In [ ]:
!pip install ai-core-sdk

In [1]:
!pip install python-dotenv

In [5]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
import os

In [39]:
# Reads .env file from the same folder
from dotenv import load_dotenv
load_dotenv()


True

In [24]:
ai_core_client = AICoreV2Client(
# `AI_API_URL`
   base_url = os.getenv("BASEURL"),
# `URL`
   auth_url =  os.getenv("AUTHURL"),
# `clientid`
  client_id = os.getenv("CLIENTID"),
# `clientsecret`
  client_secret = os.getenv("CLIENTSECRET")
)

## List Resource Groups

In [2]:
response = ai_core_client.resource_groups.query()
for app in response.resources:
    print(app.__dict__)

{'resource_group_id': 'vikrambhat', 'labels': [], 'status': 'PROVISIONED', 'created_at': datetime.datetime(2023, 7, 4, 16, 5, 9)}
{'resource_group_id': 'default', 'labels': [], 'status': 'PROVISIONED', 'created_at': datetime.datetime(2023, 6, 30, 9, 6, 9)}


## Add docker Registry

In [ ]:
dockerregistrypassword=os.getenv("DOCKERREGISTRY")

In [38]:
response = ai_core_client.docker_registry_secrets.create(
    name = "vbrepo",
    data = {
        ".dockerconfigjson": "{\"auths\":{\"https://index.docker.io\":{\"username\":\"vikrambhat2\",\"password\":\""+dockerregistrypassword+"\"}}}"
    }
)

print(response.__dict__)


{'message': 'secret has been created'}


## Add Github Repo

In [40]:
github_access_token=os.getenv("GITHUBTOKEN")

In [ ]:
ai_core_client.repositories.create(
    name = "vikram-aicore-pipelines",
    url = "https://github.com/vikrambhat2/sapaicore_mlexp",
    username = "vikrambhat2",
    password = github_access_token
)

## Check github onboarding status

In [43]:
# check on-boarding status
response = ai_core_client.repositories.query()
#
for repository in response.resources:
    print('Name:', repository.name)
    print('URL:', repository.url)
    print('Status:', repository.status)

Name: vikram-aicore-pipelines
URL: https://github.com/vikrambhat2/sapaicore_mlexp
Status: RepositoryStatus.COMPLETED


### Specify the folder containing yaml file

In [74]:
yaml_path="AUTOAIEXEC"

### Read execution file(yaml) and create an application

In [139]:
## Code to delete existing application
res=ai_core_client.applications.delete(application_name="aicore-app-autoai")
res.message

'Application has been successfully deleted.'

In [140]:
github_app_name="aicore-app-autoai"
ai_core_client.applications.create(
    application_name = github_app_name,
    repository_url = "https://github.com/vikrambhat2/sapaicore_mlexp",
    path = yaml_path,
    revision = "HEAD"
)


In [141]:
response = ai_core_client.applications.query()

for app in response.resources:
    print(app.__dict__)

{'path': 'DemandResponse', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app'}
{'path': 'AUTOAIEXEC', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-autoai'}
{'path': 'DemandResponseExec', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-vb'}
{'path': 'DemandResponse', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-vb1'}
{'path': 'DemandResponseExec2', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-vb2'}
{'path': 'DemandResponse', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-vb3'}


### Check if execution file sync is successful

In [142]:
response = ai_core_client.applications.get_status(application_name=github_app_name)


for workflow_sync_status in response.sync_ressources_status:
    print(workflow_sync_status.__dict__)
print(response.__dict__)
print('*'*80)
print(response.sync_ressources_status[0].__dict__)


{'name': 'auto-pipeline', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/auto-pipeline created'}
{'health_status': 'Healthy', 'sync_status': 'Synced', 'message': 'successfully synced (all tasks run)', 'source': <ai_core_sdk.models.application_source.ApplicationSource object at 0x7fb5485eb4f0>, 'sync_finished_at': '2023-07-16T14:55:10Z', 'sync_started_at': '2023-07-16T14:55:09Z', 'reconciled_at': '2023-07-16T14:55:10Z', 'sync_ressources_status': [<ai_core_sdk.models.application_resource_sync_status.ApplicationResourceSyncStatus object at 0x7fb518439d30>]}
********************************************************************************
{'name': 'auto-pipeline', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/auto-pipeline created'}


## Create Configuration

In [143]:
exec_id="auto-pipeline"
scen_id="demandresponse-autoai"

In [144]:
from ai_core_sdk.models import InputArtifactBinding
from ai_api_client_sdk.models.parameter_binding import ParameterBinding

response = ai_core_client.configuration.create(
    name = "demandresponse-autoai-withaicore",
    resource_group = "default",
    scenario_id = scen_id, # value from workflow
    executable_id = exec_id,
#        input_artifact_bindings = [
#        # list
#        InputArtifactBinding(key="demandresponsemodel", artifact_id =artifact_id), # Change artifact id
#    ]

)

print(response.__dict__)

{'id': 'c3b21057-f75b-4e40-a6d9-13c6df94061a', 'message': 'Configuration created'}


In [145]:
conf_id=response.__dict__['id']

In [146]:
# Lists all configurations
response = ai_core_client.configuration.query(
    resource_group = "default"
)

for configuration in response.resources:
    print(configuration.__dict__)
    


{'id': 'c3b21057-f75b-4e40-a6d9-13c6df94061a', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'executable_id': 'auto-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2023, 7, 16, 14, 55, 18), 'scenario': None}
{'id': 'a6bc9ee6-9a4e-4334-b04a-9b0bccbb6e6f', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'executable_id': 'auto-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2023, 7, 16, 14, 40, 46), 'scenario': None}
{'id': '1d747eb5-15a4-42b3-b28f-958041354f5f', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'executable_id': 'auto-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2023, 7, 16, 14, 34, 10), 'scenario': None}
{'id': '07f20b04-b89c-42ce-80c9-cfaab8f8042f', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 

## Create Deployment

In [147]:
response = ai_core_client.deployment.create(
    resource_group = 'default',
    configuration_id = conf_id # change this
)
print(response.__dict__)
deploymentid=response.__dict__['id']

{'id': 'db1da51d03258bed', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


## Check deployment status

In [155]:
response = ai_core_client.deployment.get(
    resource_group = 'default',
    deployment_id = deploymentid # Change this
)

print("Status: ", response.status)
print('*'*80)
print(response.__dict__)


Status:  Status.RUNNING
********************************************************************************
{'id': 'db1da51d03258bed', 'configuration_id': 'c3b21057-f75b-4e40-a6d9-13c6df94061a', 'configuration_name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'status': <Status.RUNNING: 'RUNNING'>, 'target_status': <TargetStatus.RUNNING: 'RUNNING'>, 'created_at': datetime.datetime(2023, 7, 16, 14, 55, 27), 'modified_at': datetime.datetime(2023, 7, 16, 14, 57, 55), 'status_message': None, 'status_details': None, 'submission_time': datetime.datetime(2023, 7, 16, 14, 56, 5), 'start_time': datetime.datetime(2023, 7, 16, 14, 57, 33), 'completion_time': None, 'deployment_url': 'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/db1da51d03258bed', 'last_operation': <Operation.CREATE: 'CREATE'>, 'latest_running_configuration_id': 'c3b21057-f75b-4e40-a6d9-13c6df94061a', 'ttl': None}


In [154]:
from ai_core_sdk.models import TargetStatus
response=ai_core_client.deployment.query_logs(resource_group = 'default',
    deployment_id = deploymentid)
for log in response.data.result:
    print(log.msg)
    print("---")

Starting
---
[2023-07-16 14:56:24 +0000] [7] [INFO] Starting gunicorn 20.1.0
---
[2023-07-16 14:56:24 +0000] [7] [INFO] Listening at: http://0.0.0.0:7000 (7)
---
[2023-07-16 14:56:24 +0000] [7] [INFO] Using worker: sync
---
[2023-07-16 14:56:24 +0000] [8] [INFO] Booting worker with pid: 8
---


## Make prediction

In [157]:
import requests

# URL
deployment_url = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/db1da51d03258bed"

# Preparing the input for inference
test_input = {"Headers": ["CUSTOMER_ID", "CSTFNM", "CSTLNM", "CSTPH1", "CSTML","AGE", "CITY", "MARITAL_STATUS", "GENDER", "EDUCATION", "EMPLOYMENT","TENURE", "SEGMENT", "HOME_SIZE", "ENERGY_USAGE_PER_MONTH","ENERGY_EFFICIENCY", "IS_REGISTERED_FOR_ALERTS", "OWNS_HOME","COMPLAINTS", "EST_INCOME", "CLTV", "HAS_THERMOSTAT","HAS_HOME_AUTOMATION", "PHOTOVOLTAIC_ZONING", "WIND_ZONING","SMART_METER_COMMENTS", "IS_CAR_OWNER", "HAS_EV", "HAS_PHOTOVOLTAIC","HAS_WIND", "EBILL", "IN_WARRANTY", "STD_YRLY_USAGE","MISSED_PAYMENT", "YEARLY_USAGE_PREDICTED"],"Input": [[7, "Elissa", "Corkery", "757-680-4340", "Elissa.Corkery@garret.net", 19, "Santa Clara", "S", "male", "High school graduate", "Employed full-time", 45, "Budget Payment Plan Members", 2330, 4680, 0.498, False, True, False, 49910, 175, True, False, False, True, "Negative", True, False, False, False, True, True, 48148, False, 45259.1]]}

endpoint = f"{deployment_url}/v2/predict" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "default",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)

print( response.json())

{'predictions': [{'prediction': 0.0, 'probability': [0.9097996143111211, 0.09020038568887895]}]}


In [158]:
import requests

# URL
deployment_url = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/db1da51d03258bed"


endpoint = f"{deployment_url}/v2/greet" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "default"}
response = requests.get(endpoint, headers=headers)

print(response.text)

Model is loaded.
